In [1]:
import helper
import os
import glob
import cv2
import matplotlib.pyplot as plt
import random
import torch

dataset_path = "archive\\dataset1000\\"
if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)

images_path = "archive\\extracted_images\\"
background_path = "archive\\background_images\\blackboard\\*.jpg"
labels = os.listdir(images_path)
backgrounds = [cv2.imread(file) for file in glob.glob(background_path)]
images = {}
for i, j, k in os.walk(os.getcwd()):
    if images_path in i:
        images[i[-1]] = list(map(lambda x:i + '\\' + x, k))    

filename = 0
while len(os.listdir(dataset_path))-1 <= 5:
    for background in backgrounds:
        coordinaten = torch.zeros((30, 4))
        for i in range(30):
            label_int = random.randint(0, len(labels)-1)
            label = labels[label_int]

            img_int = random.randint(0, len(images[label])-1)
            image = cv2.imread(images[label][img_int])

            background, image, mask = helper.prepare_background_image(background, image)

            if i == 0:
                co = torch.tensor(helper.provide_random_coordinates(background, image)).reshape((1,4))
            else:
                co = torch.tensor(helper.provide_random_coordinates(background, image)).reshape((1,4))
                while not helper.overlap(co, coordinaten):
                    co = torch.tensor(helper.provide_random_coordinates(background, image)).reshape((1,4))
                
            coordinaten[i] = co
            print(coordinaten[5, :])



            background, x, y, width, height = helper.place_image_on_background(mask, image, background, co[0].tolist())

            with open(dataset_path + "label.txt", 'a') as label_txt:
                label_txt.write(str(filename) + ".jpg, ") # dataset_path + 
                label_txt.write(str(label) + ', ')
                label_txt.write(str(x) + ', ')
                label_txt.write(str(y) + ', ')
                label_txt.write(str(width) + ', ')
                label_txt.write(str(height) + '\n')
            label_txt.close()

        cv2.imwrite(dataset_path + str(filename) + ".jpg", background)
        filename += 1
        if len(os.listdir(dataset_path))-1 >= 5:
            break




ValueError: empty range for randrange() (288, 163, -125)

In [ ]:
coordinaten = torch.zeros((30, 4))
c = 1

In [ ]:
t = backgrounds[0]
img = cv2.imread(images['5'][0])

if torch.all(coordinaten[0] == 0):
    coordinaten[0] = torch.tensor(helper.provide_random_coordinates(t, img)).reshape((1,4))
else:
    co = torch.tensor(helper.provide_random_coordinates(t, img)).reshape((1,4))
    # top left corners of all combinations
    lt = torch.max(co[:, None, :2], coordinaten[:, :2])  # [N,M,2]

    # bottom right corners of all combinations
    rb = torch.min(co[:, None, 2:], coordinaten[:, 2:])  # [N,M,2]

    # width and hight of overlap area of boxes1 and boxes2.
    wh = (rb - lt).clamp(min=0)  # [N,M,2]

    # Overlap area H * W.
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]
    if torch.all(inter == 0):
        coordinaten[c] = co
        c += 1
    else:
        print('Überscheindung')
        print(inter)
        pass


In [ ]:
# top left corners of all combinations
lt = torch.max(co[:, None, :2], coordinaten[:, :2])  # [N,M,2]

# bottom right corners of all combinations
rb = torch.min(co[:, None, 2:], coordinaten[:, 2:])  # [N,M,2]

# width and hight of overlap area of boxes1 and boxes2.
wh = (rb - lt).clamp(min=0)  # [N,M,2]

# Overlap area H * W.
inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]